In [79]:
import sys
sys.path.append('../')

import pennylane as qml
import pennylane.numpy as np 
from main.measurements import *
from main.operators import *

In [80]:
num_wires = 5
dev = qml.device( "default.qubit", 
                    wires=num_wires, shots=100)

In [81]:
X, P = X_and_P_ops( range(num_wires), -5, 5, False )    

In [82]:
fun_X = qml.s_prod( 0.5, X.pow(2) ) 
fun_P = qml.s_prod( 0.5, P.pow(2) )
H = [fun_X, fun_P]

In [83]:
class expval_XP(SampleMeasurement, StateMeasurement):

    @property
    def return_type(self):
        return Expectation

    @property
    def numeric_type(self):
        return float

    def shape(self, device, shots):
        if not shots.has_partitioned_shots:
            return ()
        num_shot_elements = sum(s.copies for s in shots.shot_vector)
        return tuple(() for _ in range(num_shot_elements))

    def process_samples(
        self,
        samples: Sequence[complex],
        wire_order: Wires,
        shot_range: Tuple[int] = None,
        bin_size: int = None,
    ):
        # print(samples)

        # estimate the ev
        # This also covers statistics for mid-circuit measurements manipulated using
        # arithmetic operators
        eigvals = qml.math.asarray(self.obs.eigvals(), dtype="float64")
        with qml.queuing.QueuingManager.stop_recording():
            prob = qml.probs(wires=self.wires
                                ).process_samples(samples=samples, 
                                                    wire_order=wire_order, 
                                                    shot_range=shot_range, 
                                                    bin_size=bin_size
                                                    )
        return qml.math.dot(prob, eigvals)
    
    def process_state(self, 
                        state: Sequence[complex], 
                        wire_order: Wires):
        # print(state)
        # This also covers statistics for mid-circuit 
        # measurements manipulated using
        # arithmetic operators
        eigvals = qml.math.asarray(self.obs.eigvals(), 
                                    dtype="float64")
        # we use ``self.wires`` instead of ``self.obs`` 
        # because the observable was
        # already applied to the state
        with qml.queuing.QueuingManager.stop_recording():
            prob = qml.probs(wires=self.wires
                            ).process_state(state=state, 
                                            wire_order=wire_order)
        # In case of broadcasting, `prob` has two axes 
        # and this is a matrix-vector product
        return qml.math.dot(prob, eigvals)

In [84]:
def init_state():
    # qml.MottonenStatePreparation(min_state, wires=range(num_wires) )
    for k in range(num_wires):
        qml.RX(np.pi/5,k)
        qml.RZ(np.pi/3,k)

In [85]:
def exp_val_XP( init_state, local_hamils, device ):
    
    if not isinstance( local_hamils, list ):
        local_hamils = [local_hamils]
        
    circuit_tapes = []
    eigenvals     = []
    
    for local_hamil in local_hamils :

        with qml.tape.OperationRecorder() as circuit_tape:
            init_state()

        circuit_tape = circuit_tape.operations
        circuit_tape =  circuit_tape + local_hamil.diagonalizing_gates()

        circuit_tape = qml.tape.QuantumTape( ops= circuit_tape,
                                measurements=[ qml.probs(wires=local_hamil.wires) ] )
        
        circuit_tapes.append( circuit_tape )
        eigenvals.append( local_hamil.eigvals() )

    probs = qml.execute( circuit_tapes, 
                        device,
                        gradient_fn=qml.gradients.param_shift )

    ExpVal = 0
    for j, prob in enumerate(probs):
        ExpVal += np.dot( eigenvals[j], prob)

    return ExpVal


In [86]:
exp_val_XP( init_state, fun_X, dev )

9.713778203124788

In [87]:
exp_val_XP( init_state, fun_P, dev )

10.759895143437808

In [88]:
exp_val_XP( init_state, H, dev )

20.473673346562595

In [211]:
num_wires = 5
dev = qml.device( "default.qubit", 
                    wires=2*num_wires, shots=100)

In [224]:
def init_state(theta=0):
    for k in range(2*num_wires):
        qml.RX(np.pi/5,k)
        qml.RZ(np.pi/3,k)
    qml.RY(theta,0)

In [225]:
X1, P1 = X_and_P_ops( range(num_wires), -5, 5, False )    
X2, P2 = X_and_P_ops( range(num_wires,2*num_wires), -5, 5, False )    

H12 = [ 
        X1.pow(2),
        X2.pow(2),
        P1.pow(2),
        P2.pow(2),
        qml.prod( X1, X2 )
        ]
H12 

[<Hamiltonian: terms=1, wires=[0, 1, 2, 3, 4]>,
 <Hamiltonian: terms=1, wires=[5, 6, 7, 8, 9]>,
 <Hamiltonian: terms=1, wires=[0, 1, 2, 3, 4]>,
 <Hamiltonian: terms=1, wires=[5, 6, 7, 8, 9]>,
   (1) [Hermitian0,1,2,3,4] @   (1) [Hermitian5,6,7,8,9]]

In [251]:
def exp_val_XP( theta ):
    
    device       = dev
    local_hamils = H12
    
    if not isinstance( local_hamils, list ):
        local_hamils = [local_hamils]
        
    circuit_tapes = []
    eigenvals     = []
    
    # ExpVal = 0
    for local_hamil in local_hamils :

        with qml.tape.OperationRecorder() as circuit_tape:
            init_state(theta[0])

        circuit_tape = circuit_tape.operations
        circuit_tape =  circuit_tape + local_hamil.diagonalizing_gates()

        circuit_tape = qml.tape.QuantumTape( ops= circuit_tape,
                                measurements=[ qml.probs(wires=local_hamil.wires) ] )
        
        circuit_tapes.append( circuit_tape )
        eigenvals.append( local_hamil.eigvals() )

    probs = qml.execute( circuit_tapes, 
                        device,
                        gradient_fn=qml.gradients.param_shift )

    ExpVal = 0
    for j, prob in enumerate(probs):
        ExpVal += np.dot( eigenvals[j], prob)

    return ExpVal


In [252]:
theta = np.array([1.], requires_grad=True) 
exp_val_XP( theta ) 


82.09319450576879

In [253]:
qml.grad(exp_val_XP)( np.array([1.], requires_grad=True) )

array([-19.62778855])